In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ShuffleSplit
from sklearn.neural_network import MLPClassifier
from sklearn.base import BaseEstimator
import torchmetrics.functional as metrics
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import os
from typing import List

In [2]:
import sys
sys.path.append('../')

In [3]:
from classes.Patient import  Patient
from classes.Dataset import Dataset

In [4]:
x=0
m=0
path = Path('../data')
for patient_dir in os.listdir(path):
    if patient_dir == "sigma":
        continue
    m+=1
    x += len(os.listdir(Path.joinpath(path, patient_dir)))
    print(f"\'{Path.joinpath(path, patient_dir)}\',#", len(os.listdir(Path.joinpath(path, patient_dir))))
    
                             
path = Path('../data/sigma')
for patient_dir in os.listdir(path):
    x += len(os.listdir(Path.joinpath(path, patient_dir)))
    m+=1
    print(f"\'{Path.joinpath(path, patient_dir)}\',#", len(os.listdir(Path.joinpath(path, patient_dir))))
    

print(x,m)


'..\data\Healthy_15M_4753',# 135
'..\data\Healthy_15M_4754',# 133
'..\data\Healthy_4M_4240',# 95
'..\data\Healthy_4M_4242',# 106
'..\data\Healthy_4M_4243',# 146
'..\data\Healthy_8M_4207',# 69
'..\data\Healthy_8M_4208',# 122
'..\data\Healthy_8M_4209',# 88
'..\data\Sick_15M_4270',# 155
'..\data\Sick_15M_4272',# 118
'..\data\Sick_15M_4771',# 63
'..\data\Sick_4M_4255',# 124
'..\data\Sick_4M_4256',# 168
'..\data\Sick_8M_3230',# 170
'..\data\Sick_8M_3231',# 56
'..\data\sigma\Healthy_15M_4756',# 62
'..\data\sigma\Healthy_4M_4240',# 107
'..\data\sigma\Healthy_4M_4241',# 37
'..\data\sigma\Healthy_4M_42411',# 120
'..\data\sigma\Healthy_8M_4207',# 90
'..\data\sigma\Healthy_8M_4208',# 85
'..\data\sigma\Healthy_8M_4209',# 134
'..\data\sigma\Sick_15M_4272',# 89
'..\data\sigma\Sick_4M_4253',# 146
'..\data\sigma\Sick_4M_4254',# 153
'..\data\sigma\Sick_4M_4255',# 110
'..\data\sigma\Sick_4M_4256',# 82
'..\data\sigma\Sick_8M_3230',# 167
'..\data\sigma\Sick_8M_3231',# 113
3243 29


In [5]:
patient_paths = [
'..\data\sigma\Healthy_15M_4756',# 62
'..\data\sigma\Healthy_4M_4240',# 107
'..\data\sigma\Healthy_4M_4241',# 37
'..\data\sigma\Healthy_4M_42411',# 120
'..\data\sigma\Healthy_8M_4207',# 90
'..\data\sigma\Healthy_8M_4208',# 85
'..\data\sigma\Healthy_8M_4209',# 134
'..\data\sigma\Sick_15M_4272',# 89
'..\data\sigma\Sick_4M_4253',# 146
'..\data\sigma\Sick_4M_4254',# 153
'..\data\sigma\Sick_4M_4255',# 110
'..\data\sigma\Sick_4M_4256',# 82
'..\data\sigma\Sick_8M_3230',# 167
'..\data\sigma\Sick_8M_3231',# 113
]



In [6]:
patients = [Patient(path_main_dir) for path_main_dir in patient_paths]
for patient in patients:
    patient.load_data()

100%|██████████| 113/113 [00:10<00:00, 10.40it/s]


In [7]:
X = np.array(patients)
y = np.array([x._class for x in patients])
sss = ShuffleSplit(n_splits=20, test_size=0.33, random_state=0)


In [8]:
def preprocess(patients: np.ndarray, y: np.ndarray):
    patients_dataset = Dataset(patients)
    patients_dataset.concat_data()
    
    _df = pd.concat([x for x in patients_dataset.summary_image.summary_image_data])
    y = _df['class']

    _df['avg_dendrite_area'] = _df.apply(lambda x: x['AreaOccupied_AreaOccupied_secondary_dendrities'] / x['Count_secondary_dendrities'] if x['Count_secondary_dendrities'] != 0 else 0, axis=1)
    _df = _df.drop(columns=['patient_path','Metadata_FileLocation','Frame_input_illum','FileName_input_illum','PathName_input_illum','URL_input_illum','class','ImageSet_ImageSet','MD5Digest_input_illum']).fillna(0)

    undersample = RandomUnderSampler(sampling_strategy='majority')
    X_df, y = undersample.fit_resample(_df, y)
    return X_df, y

def get_cross_valid_scaled_datasets(sss: ShuffleSplit, X, y):
    for i, (train_index, test_index) in enumerate(sss.split(X, y)):
        train_patients, y_train = X[train_index], y[train_index] 
        test_patients, y_test = X[test_index], y[test_index] 
       
        if all(x == 0 for x in y_train) \
            or all(x == 1 for x in y_train) \
            or all(x == 0 for x in y_test) \
            or all(x == 1 for x in y_test):
            continue
        
        X_train, y_train = preprocess(train_patients, y_train)
        X_test, y_test = preprocess(test_patients, y_test)
        
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        yield X_train_scaled, y_train, X_test_scaled, y_test 
        
for X_train, y_train, X_test, y_test in get_cross_valid_scaled_datasets(sss, X, y):
    print(np.shape(X_train), np.shape(y_train), np.shape(X_test), np.shape(y_test))

(748, 93) (748,) (456, 93) (456,)
(606, 93) (606,) (164, 93) (164,)
(982, 93) (982,) (288, 93) (288,)
(578, 93) (578,) (164, 93) (164,)
(548, 93) (548,) (456, 93) (456,)
(982, 93) (982,) (288, 93) (288,)
(916, 93) (916,) (342, 93) (342,)
(780, 93) (780,) (268, 93) (268,)
(822, 93) (822,) (448, 93) (448,)
(976, 93) (976,) (294, 93) (294,)
(706, 93) (706,) (526, 93) (526,)
(582, 93) (582,) (178, 93) (178,)
(788, 93) (788,) (482, 93) (482,)
(776, 93) (776,) (390, 93) (390,)
(928, 93) (928,) (342, 93) (342,)
(662, 93) (662,) (226, 93) (226,)
(752, 93) (752,) (518, 93) (518,)
(854, 93) (854,) (304, 93) (304,)
(582, 93) (582,) (292, 93) (292,)


In [9]:
class Net(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 48)
        self.fc2 = nn.Linear(48, 24)
        self.fc3 = nn.Linear(24, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.x, self.y = dataset        
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        X = self.x[idx]
        y = self.y[idx]
        return X, y

def train_eval_torch_model(model, X_train, y_train, X_test, y_test):
    train_dataset = MyDataset((X_train, y_train.to_numpy()))
    test_dataset = MyDataset((X_test, y_test.to_numpy()))
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=16)
    testloader = torch.utils.data.DataLoader(test_dataset, batch_size=16)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    #TRAIN
    for epoch in range(20):  # loop over the dataset multiple times
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            # zero the parameter gradients
            optimizer.zero_grad()
            inputs = inputs.to(torch.float32)
            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            # print(f'[{epoch + 1}, {i + 1:5d}] loss: {loss.item() :.3f}')

    #EVAL
    predictions = []
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.to(torch.float32)

            # calculate outputs by running images through the network
            outputs = model(images)
            # print(outputs.data)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            # print(predicted)
            predictions.append(predicted.cpu().detach().numpy().tolist())

    pred_flatten = [item for row in predictions for item in row]
    return torch.Tensor(pred_flatten)


def train_eval_sklearn_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    return torch.Tensor(model.predict(X_test))

class MyModel:
    def __init__(self, model) -> None:
        self.model = model
    
    def train_eval_model(self):
        if isinstance(self.model, Net):
            return train_eval_torch_model
        elif isinstance(self.model, BaseEstimator):
            return train_eval_sklearn_model
        else:
            raise ValueError("Invalid model")
        

    
class Evaluator:
    def __init__(self, X:np.ndarray(shape=len(X),dtype=Patient), y: np.ndarray(shape=len(y),dtype=str), n_splits: int):
        self.X = X
        self.y = y
        self.ss = ShuffleSplit(n_splits=n_splits, test_size=0.33, random_state=0)
        

    def prepare_models(self, num_features:int):
        models = [
            LogisticRegression(penalty='l1',solver='saga', C=0.7, max_iter=1000),
            MLPClassifier(solver='sgd', alpha=1e1, hidden_layer_sizes=(24,12), random_state=1,verbose=False, learning_rate="adaptive",learning_rate_init=0.001,early_stopping=True),
            Net(num_features)
        ]
        self.models :List[MyModel] = [MyModel(model) for model in models]
    
    def prepare_metrics(self):
        self.metrics_values = {}
        self.metrics = [metrics.accuracy, metrics.recall, metrics.precision, metrics.f1_score] #, metrics.roc]
        self.main_metric = metrics.accuracy.__name__
        for model in self.models:
            model_name = model.model.__name if hasattr(model, '__name__') else model.model.__class__.__name__

            self.metrics_values[model_name] = {metric.__name__ if hasattr(metric, '__name__') else metric.__class__.__name__: [] for metric in self.metrics}


    def calculate_metrics(self, model: str, y_true: torch.Tensor, y_pred: torch.Tensor) -> None:
        for metric in self.metrics:
            metric_name = metric.__name__ if hasattr(metric, '__name__') else metric.__class__.__name__
            model_name = model.__name if hasattr(model, '__name__') else model.__class__.__name__

            value = metric(torch.squeeze(y_pred), y_true.int(),"binary")
            self.metrics_values[model_name][metric_name].append(value.item())


    def run(self):
        self.prepare_models(0)
        self.prepare_metrics()

        for X_train, y_train, X_test, y_test in get_cross_valid_scaled_datasets(self.ss, self.X, self.y):
            print(np.shape(X_train), np.shape(y_train), np.shape(X_test), np.shape(y_test))
            num_features = np.shape(X_train)[1]
            self.prepare_models(num_features)
            
            for model in self.models:
                train_fn = model.train_eval_model()
                y_pred = train_fn(model.model, X_train, y_train, X_test, y_test)
                self.calculate_metrics(model.model, torch.Tensor(y_test.values), y_pred)
            



In [10]:
evaluator = Evaluator(X, y, 20)
evaluator.run()

d:\mainD\study\toolsTmp\myAnaconda\Anaconda\envs\codee\lib\site-packages\torch\nn\init.py:412: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


(748, 93) (748,) (456, 93) (456,)


d:\mainD\study\toolsTmp\myAnaconda\Anaconda\envs\codee\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


(606, 93) (606,) (164, 93) (164,)


d:\mainD\study\toolsTmp\myAnaconda\Anaconda\envs\codee\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


...

d:\mainD\study\toolsTmp\myAnaconda\Anaconda\envs\codee\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


(854, 93) (854,) (304, 93) (304,)


d:\mainD\study\toolsTmp\myAnaconda\Anaconda\envs\codee\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


(582, 93) (582,) (292, 93) (292,)


d:\mainD\study\toolsTmp\myAnaconda\Anaconda\envs\codee\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [11]:
avg_metrics = {}
for model_name, model_metrics in evaluator.metrics_values.items():
    avg_metrics[model_name] = {}
    for metric_name, metric_value in model_metrics.items():
        avg_metrics[model_name][metric_name] = np.mean(metric_value)
print(pd.DataFrame(data=avg_metrics))

           LogisticRegression  MLPClassifier       Net
accuracy             0.746802       0.703850  0.748494
recall               0.782756       0.734956  0.766672
precision            0.756627       0.700312  0.775184
f1_score             0.748307       0.696657  0.742820
